<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

* Импортируем библиотеки
* Посмотрим файл
* Предобработаем данные
* Выделим целевой признак и обучающие
* Проведём лемматизацию, очистку и векторизацию данных

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from tqdm import tqdm
from nltk.corpus import stopwords as nltk_stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [5]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
lemmatizer = WordNetLemmatizer()

In [7]:
def lemmatization(text):
    word_list = nltk.word_tokenize(text)
    pos_tags = pos_tag(word_list)
    lemmatized_words = []
    for word, pos in pos_tags:
        # Применение лемматизации с учетом POS-тега
        if pos.startswith('NN'):  # Существительное
            lemma = lemmatizer.lemmatize(word, pos='n')
        elif pos.startswith('VB'):  # Глагол
            lemma = lemmatizer.lemmatize(word, pos='v')
        elif pos.startswith('JJ'):  # Прилагательное
            lemma = lemmatizer.lemmatize(word, pos='a')
        elif pos.startswith('R'):  # Наречие
            lemma = lemmatizer.lemmatize(word, pos='r')
        else:
            lemma = lemmatizer.lemmatize(word)  # По умолчанию
        lemmatized_words.append(lemma)
    return ' '.join(lemmatized_words)

Напишем функцию, которая приводит текст к нижнему регистру, удаляет все знаки, разбивает тексты на отдельные слова и соединяет их обратно в строки. 

In [8]:
def clear_text(text):
    return ' '.join(re.sub(r'[^a-z]', ' ', text.lower()).split())

In [9]:
tqdm.pandas()
data = data[:5]
data['text'] = data['text'].progress_apply(lemmatization)
data['text'] = data['text'].progress_apply(clear_text)

100%|██████████| 159292/159292 [00:07<00:00, 21725.72it/s]


In [10]:
features = data['text']
target = data['toxic']

Разделим данные на обучающую и тестовую выборки в соотношении 50 : 50

In [11]:
target_train, target_test, features_train, features_test = train_test_split(target, features, test_size=0.5,  
                                                                            random_state=42)

In [12]:
features_train[:5]

58623                           january fine then i ll stop
147333    noteable under point of the academic notabilit...
69872     here be the only thing i can find regarding us...
1552      utc the npov tag have be restore because of re...
29797     i dont however gun powder ma would you mind st...
Name: text, dtype: object

Проведём векторизацию для тренировочной выборки

In [13]:
stop_words = set(nltk_stopwords.words('english'))
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 2))
features_train = tfidf_vectorizer.fit_transform(features_train)

Проведём векторизацию тестовой выборки

In [14]:
features_test = tfidf_vectorizer.transform(features_test)

Создадим переменную для удобства дальнейшего сравнения моделей. 

In [15]:
results = pd.DataFrame(columns=['Model', 'F1_score'])

## Обучение

Обучим и сравним модели: 
* LogisticRegression
* LinearSVC

In [16]:
%%time
model_LR = LogisticRegression(random_state=42, class_weight='balanced', solver='liblinear')
parameters = [{'C': [1, 5, 10]}]

LR_model_gscv = GridSearchCV(model_LR, param_grid=parameters, scoring='f1', cv=3, verbose=True, n_jobs=4)
LR_model_gscv.fit(features_train, target_train)

print("Лучшие параметры для LogisticRegression: ", LR_model_gscv.best_params_)
print("Оценка f1 на обучающем наборе: {:.4f}".format(LR_model_gscv.best_score_))
results.loc[len(results)] = ['LogisticRegression train', '{:.4f}'.format(LR_model_gscv.best_score_)]

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Лучшие параметры для LogisticRegression:  {'C': 10}
Оценка f1 на обучающем наборе: 0.7661
CPU times: user 17.7 s, sys: 9.39 s, total: 27.1 s
Wall time: 1min 38s


In [17]:
%%time
model_LSVC = LinearSVC(random_state=42, class_weight='balanced')

parameters = [{'C': [0.3, 0.4, 0.5]}]

LSVC_model_gscv = GridSearchCV(model_LSVC, param_grid=parameters, scoring='f1', cv=5, verbose=True, n_jobs=4)
LSVC_model_gscv.fit(features_train, target_train)

print("Лучшие параметры для LinearSVC: ", LSVC_model_gscv.best_params_)
print("Оценка f1 на обучающем наборе: {:.4f}".format(LSVC_model_gscv.best_score_))
results.loc[len(results)] = ['LinearSVC train', '{:.4f}'.format(LSVC_model_gscv.best_score_)]

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Лучшие параметры для LinearSVC:  {'C': 0.3}
Оценка f1 на обучающем наборе: 0.7704
CPU times: user 3.35 s, sys: 85.4 ms, total: 3.43 s
Wall time: 55.9 s


In [18]:
results

,Model,F1_score
0,LogisticRegression train,0.7661
1,LinearSVC train,0.7704


Как видим, с нашей задачей модель LinearSVC справляется несколько лучше, чем LogisticRegression. Будем использовать её на тестовой выборке.

## Выводы

In [20]:
# Оцениваем модель на тестовом наборе
predict_LSVC = LSVC_model_gscv.predict(features_test)
f1_LSVC = f1_score(target_test, predict_LSVC)
print("Оценка f1 на тестовом наборе: {:.4f}".format(f1_LSVC))

Оценка f1 на тестовом наборе: 0.7668


# Вывод:
* Анализ моделей проведён на тренировоной и тестовой выборках
* F1 метрика на всех тестах составляет 0.75 +
* Модель LinearSVC справляется с нашей задачей несколько лучше, чем LogisticRegression
* На обучающей выборке метрика F1 составила: LinearSVC -	0.7704, LogisticRegression - 0.7661
* На тестовой выборке метрика F1 модели LinearSVC составила 0.7668

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны